## Algoritmo implementados en el Orquestador

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import openpyxl

# Generación de la base de datos

In [2]:
def get_column_names_sheet(info):
  col = []
  for i in range(10):
    v = info.cell(row = 1, column = i+1).value
    if v != None:
      col.append(v)
  return col

In [3]:
def get_df_Dic_Data(base_datos):
  df_lis = []
  hojas = base_datos.get_sheet_names()
  for i in hojas:
    info = base_datos.get_sheet_by_name(name= i)
    v = get_column_names_sheet(info)
    par = list(zip(v,[i]*len(v)))
    df_lis+=par

  return pd.DataFrame(df_lis,columns=['campo','tabla'])

In [4]:
def create_query_table(i,p):
  query= f"CREATE TABLE IF NOT EXISTS {i}("

  for a,b in zip(p['campo'],p['dt']):
    query=query+' '+a+' '+b+','

  query = query[:-1]+')'

  return query

In [5]:
path_info_datos = 'Base de datos.xlsx'

In [6]:
base_datos = openpyxl.load_workbook(path_info_datos)

In [7]:
dic_data = get_df_Dic_Data(base_datos)

/tmp/ipykernel_5149/3143674730.py:3: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  hojas = base_datos.get_sheet_names()
/tmp/ipykernel_5149/3143674730.py:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  info = base_datos.get_sheet_by_name(name= i)


In [32]:
dic_data.to_csv('datos.csv')

In [9]:
DD1 = pd.read_excel(path_info_datos,'Tablas')
DD2 = pd.read_excel(path_info_datos,'DiccionarioDatos')

In [11]:
DD1.head()

,id_tabla,nombre,endpoint,tipo,Desc
0,1,CONTRATOS,NaN,catalogo,"catalogo de restricciones de los componentes, ..."
1,2,TRANSACCIONESAUDITOR,NaN,Transaccional,Transacciones realizadas dentro del orquestador
2,3,ARCHIVOS,NaN,Transaccional,Archivos utilizados dentro de las practicas
3,4,PRACTICAS,NaN,Catalogo,Practicas que se puede realizar dentro del lab...
4,5,COMPONENTES,NaN,Catalogo,Catalogo de componentes disponibles


In [12]:
DD2.head()

,idcampo,id_tabla,campo,tipo_dato,PK/FK,Descripcion,ejemplo
0,1,1,IdContrato,integer,PK,identificador de la restriccion,1
1,2,1,NombreContrato,string,no,nombre de la restriccion,movimiento brazo robotico
2,3,1,TipoContrato,string,no,"tipo de restriccion, es decir, si se aplica al...",maqueta
3,4,1,DescContrato,string,no,Desc detallada de la retriccion,NaN
4,5,1,EspecificacionContrato,string,no,información de formato especial para la carga,NaN


In [13]:
dic_data.head()

,campo,tabla
0,id_tabla,Tablas
1,nombre,Tablas
2,endpoint,Tablas
3,tipo,Tablas
4,Desc,Tablas


In [14]:
pd.merge(dic_data,DD1,left_on='tabla',right_on='nombre')

,campo,tabla,id_tabla,nombre,endpoint,tipo,Desc
0,id_usuario,USUARIOS,17,USUARIOS,catUser,Transaccional,Usuarios registrados en el LABREMO
1,username,USUARIOS,17,USUARIOS,catUser,Transaccional,Usuarios registrados en el LABREMO
2,email,USUARIOS,17,USUARIOS,catUser,Transaccional,Usuarios registrados en el LABREMO
3,isAvailable,USUARIOS,17,USUARIOS,catUser,Transaccional,Usuarios registrados en el LABREMO
4,usertype,USUARIOS,17,USUARIOS,catUser,Transaccional,Usuarios registrados en el LABREMO
...,...,...,...,...,...,...,...
104,idMateria,PRACTICAS,14,PRACTICAS,NaN,Catalogo,Catalogo de practicas disponibles para el LABREMO
105,createDate,PRACTICAS,4,PRACTICAS,NaN,Catalogo,Practicas que se puede realizar dentro del lab...
106,createDate,PRACTICAS,14,PRACTICAS,NaN,Catalogo,Catalogo de practicas disponibles para el LABREMO
107,SerializedSet,PRACTICAS,4,PRACTICAS,NaN,Catalogo,Practicas que se puede realizar dentro del lab...


In [30]:
DD2['tipo_dato'].unique()

array(['integer', 'string', 'list', 'bolean', 'date'], dtype=object)

In [31]:
def generate_queries_table(DD1,DD2,mape):
  DD2['dt'] = DD2['tipo_dato'].map(mape);
  queries = []
  for i in range(DD1.shape[0]):
    try:
      id,nombre = DD1[['id_tabla','nombre']].iloc[i]
      p = DD2[DD2['id_tabla']==id]
      queries.append(create_query_table(nombre,p))
    except:
      print(f'revisar {nombre}')
  return queries


In [ ]:
def create_table(queries,database):
  for i in queries:
    try :
      cur = database.cursor()
      cur.execute(i)
    except:
      print(f'revisar {i}')

In [32]:
mape = {'integer':'integer','string':'VARCHAR(255)','list':'VARCHAR(255)','bolean':'BOOLEAN','date':'DATETIME'}

In [52]:
quer = generate_queries_table(DD1,DD2,mape);


In [53]:
con = sqlite3.connect("LABREMODB.db")

In [54]:
create_table(quer,con)

revisar CREATE TABLE IF NOT EXISTS SESIONES)
revisar CREATE TABLE IF NOT EXISTS ALUMNOS)
revisar CREATE TABLE IF NOT EXISTS SESIONES)
revisar CREATE TABLE IF NOT EXISTS PROFESORES)
revisar CREATE TABLE IF NOT EXISTS COURSES)


In [56]:
cor= con.cursor()
cor.execute("""SELECT name FROM sqlite_master """)
print(cor.fetchall())

[('CONTRATOS',), ('TRANSACCIONESAUDITOR',), ('ARCHIVOS',), ('PRACTICAS',), ('COMPONENTES',), ('MAQUETA',), ('ASIGNACIONMAQUETAS',), ('SUPERVISOROPERACIONES',), ('PAISES',), ('INSTITUCIONES',), ('FACULTADES',), ('CARRERAS',), ('MATERIAS',), ('USUARIOS',), ('TIPOUSUARIO',)]


In [3]:
import json

In [59]:
f = open("/home/ffelix07/Documents/LabRemo/Orchestrator/ros2_labremoto/paths.json")

In [63]:
paths = {'path_database': '/home/ffelix07/Documents/LabRemo/Orchestrator/LABREMODB.db'}

In [65]:
with open('/home/ffelix07/Documents/LabRemo/Orchestrator/'+"VARIABLES_ORQUESTADOR.json", "w") as outfile: 
    json.dump(paths, outfile)

In [5]:
import sqlite3
import pandas as pd

In [17]:
con = sqlite3.connect("LABREMODB.db")

In [19]:
cursor = con.cursor()
query =  f""" SELECT * FROM USUARIOS WHERE Id = 1"""
cursor.execute(query)
info = cursor.fetchall()
info

[]

In [11]:
cor= con.cursor()
cor.execute("""SELECT *
             FROM SUPERVISOROPERACIONES """)
pd.DataFrame(cor.fetchall())


,0,1,2,3,4
0,None,Inicio Nodo,supervisor_operaciones,Publicado,2024-02-21 17:17:33.472196
1,None,Inicio Nodo,supervisor_operaciones,Publicado,2024-02-21 17:17:43.039071
2,None,Inicio Nodo,administrador_transacciones_entrada,Publicado,2024-02-21 17:19:07.821458


In [8]:
cor= con.cursor()
cor.execute("""SELECT *
             FROM TRANSACCIONESAUDITOR """)
pd.DataFrame(cor.fetchall())


""


In [3]:
cor= con.cursor()
cor.execute("""DELETE FROM SUPERVISOROPERACIONES """)
con.commit()
cor.execute("""DELETE FROM TRANSACCIONESAUDITOR """)
con.commit()

In [6]:
cor= con.cursor()
cor.execute(f""" SELECT * FROM CONTRATOS WHERE IdContrato = 4""")
pd.DataFrame(cor.fetchall())


,0,1,2,3,4
0,4,wef2,wef3,wef4,wef5
1,4,wef2,wef3,wef4,wef5
2,4,wef2,wef3,wef4,wef5


In [310]:
cor= con.cursor()
cor.execute(f""" SELECT * FROM TRANSACCIONESAUDITOR""")
pd.DataFrame(cor.fetchall())

,0,1,2,3,4,5
0,0,administrador_contratos,,2024-02-16 17:13:28.358145,,Transaccion aceptada


In [309]:
cor= con.cursor()
query =  f""" INSERT INTO CONTRATOS VALUES (4,'wef2','wef3','wef4','wef5')"""
cor.execute(query)
con.commit()



In [13]:
con2 = sqlite3.connect("/home/ffelix07/Documents/LabRemo/Orchestrator/2024-02-16-20-51-19/2024-02-16-20-51-19_0.db3")

In [290]:
cor= con2.cursor()
cor.execute("""SELECT name FROM sqlite_master """)
cor.fetchall()

[('schema',), ('metadata',), ('topics',), ('messages',), ('timestamp_idx',)]

In [292]:
cor= con2.cursor()
cor.execute("""SELECT * FROM messages """)
pd.DataFrame(cor.fetchall(),columns= [x[0] for x in cor.description])

,id,topic_id,timestamp,data


OperationalError: no such table: USUARIOS

In [16]:
cor= con2.cursor()
cor.execute("""SELECT name FROM sqlite_master """)
print(cor.fetchall())

[('schema',), ('metadata',), ('topics',), ('messages',), ('timestamp_idx',)]
